<a href="https://colab.research.google.com/github/aghadavood/NLP/blob/main/QAPersian_bert_base_multilingual_cased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'strart:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5220742%2F8704161%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240618%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240618T220106Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D29616bd34a44e2a811a02833214cf6144ff71b17c1e3be29303de10f14e8ed1fdc8a4c8e1885df4d2c5eb2d9adf470b84d45de0d10648b2f9642e8471e943d77365d66cecba9e6088f82aa38e05b9c328d041b98d576f3b87aedae70b116645140dc7910bddc5c500c7e3e252cfb07ca6d4c7c4ad00888e21db971ccdf47bd9cbdc7f75cf86079000da1529be7134c112657bf15cb4a82d34720dd1ed448166f8da650621b784f67c5607df170c3f3a05a5a35a28c43e880c4abd4511ef758b0e50c2e70fb540f9b08217371d33050fa2023b011aa67fd3c4710fefaf6204b12f6e954fa4784f166d91485a249ad52c68ab270b689733cb04bddf4ac79d1dd98'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
from transformers import BertTokenizerFast

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

def tokenize_dataset(dataset, tokenizer):
    def preprocess_function(examples):
        questions = examples['question']
        contexts = examples['context']
        answers = examples['answers']

        inputs = tokenizer(questions, contexts, truncation=True, padding='max_length', max_length=512)

        start_positions = []
        end_positions = []

        for i in range(len(questions)):
            context = contexts[i]
            if len(answers[i]['text']) == 0 or len(answers[i]['answer_start']) == 0:
                start_positions.append(0)
                end_positions.append(0)
                continue

            answer = answers[i]['text'][0]
            answer_start = answers[i]['answer_start'][0]
            answer_end = answer_start + len(answer)

            # Tokenize context with the fast tokenizer
            tokenized_context = tokenizer(context, truncation=True, padding='max_length', max_length=512)

            # Find start and end token positions
            start_token_idx = tokenized_context.char_to_token(answer_start)
            end_token_idx = tokenized_context.char_to_token(answer_end - 1)

            # Handle edge case where char_to_token returns None
            if start_token_idx is None:
                start_token_idx = 0
            if end_token_idx is None:
                end_token_idx = len(tokenized_context.input_ids) - 1

            start_positions.append(start_token_idx)
            end_positions.append(end_token_idx)

        inputs.update({'start_positions': start_positions, 'end_positions': end_positions})
        return inputs

    tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    return tokenized_datasets

# Load the dataset
from datasets import load_dataset
dataset = load_dataset("SajjadAyoubi/persian_qa")

# Tokenize the dataset
tokenized_datasets = tokenize_dataset(dataset, tokenizer)
print(tokenized_datasets)


In [ ]:
from transformers import BertForQuestionAnswering, Trainer, TrainingArguments

# Load the model
model = BertForQuestionAnswering.from_pretrained('bert-base-multilingual-cased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Save the model and tokenizer
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


In [ ]:
import random

# Check tokenized dataset
sample = random.choice(tokenized_datasets['train'])
print("Sample Question:", tokenizer.decode(sample['input_ids']))
print("Start Position:", sample['start_positions'])
print("End Position:", sample['end_positions'])
print("Answer Tokens:", tokenizer.decode(sample['input_ids'][sample['start_positions']:sample['end_positions'] + 1]))


In [ ]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch

# Load the fine-tuned model and tokenizer
model = BertForQuestionAnswering.from_pretrained('./fine_tuned_model')
tokenizer = BertTokenizerFast.from_pretrained('./fine_tuned_model')


Step 2: Define a Function to Predict Answers


In [ ]:
def predict_answer(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)
    outputs = model(**inputs)
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer with the argmax of the score
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1

    input_ids = inputs['input_ids'][0]
    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[start_index:end_index])
    answer = tokenizer.convert_tokens_to_string(answer_tokens)

    return answer




Step 3: Test the Model with Sample Inputs


check step by step , becasue the model return cls instead of answering right